# User Ingestion Application
### CSV Validation & API Upload Tool

**Workflow:**
1. Validates CSV against schema rules
2. **If ALL pass** → Auto-uploads to API
3. **If ANY fail** → Shows error report (downloadable)

---

### CSV Validation Rules Guide

Before preparing your CSV, review the validation rules below. Every row is checked against these rules — any failure blocks the entire upload.

---

#### Header Validation
- CSV must contain **exactly 16 columns** with these exact names (no extra, no missing):

`username`, `password`, `name`, `mobile_number`, `gender`, `date_of_birth`, `email`, `correspondence_address`, `roles`, `employment_type`, `date_of_joining`, `department`, `designation`, `campaign_name`, `administrative_area`, `boundary_code`

---

#### Field-Level Validation Rules

| # | Column | Required | Unique | Rule | Allowed Values / Format |
|---|--------|----------|--------|------|-------------------------|
| 1 | `username` | Yes | Yes | Letters, numbers, hyphens, underscores only. **No `/` or `\`** | 1–100 chars, pattern: `^[^/\\]+$` |
| 2 | `password` | Yes | No | Must contain at least **one uppercase letter** | 1–100 chars, pattern: `.*[A-Z].*` |
| 3 | `name` | Yes | No | **Only alphabets and spaces** — no numbers or special characters | 1–200 chars, pattern: `^[A-Za-z ]+$` |
| 4 | `mobile_number` | No | Yes | **8 to 15 digits only** (if provided) | Pattern: `^[0-9]{8,15}$` |
| 5 | `gender` | No | No | Must match one of the allowed values (case-insensitive) | `MALE`, `FEMALE`, `OTHER` |
| 6 | `date_of_birth` | No | No | Date format with valid day/month ranges | `DD/MM/YYYY` or `DD-MM-YYYY` |
| 7 | `email` | No | No | Standard email format (if provided) | Max 200 chars, e.g. `user@example.com` |
| 8 | `correspondence_address` | No | No | Free text | Max 500 chars |
| 9 | `roles` | Yes | No | Comma-separated role names. Each must exist in `rolesmapping.json` | See valid roles list below |
| 10 | `employment_type` | Yes | No | Must be one of the allowed values (exact match) | `TEMPORARY`, `PERMANENT` |
| 11 | `date_of_joining` | Yes | No | Date format, must be a valid calendar date | `DD/MM/YYYY` or `DD-MM-YYYY` |
| 12 | `department` | Yes | No | Fixed value | `OTHER` |
| 13 | `designation` | No | No | Free text | Max 100 chars |
| 14 | `campaign_name` | Yes | No | Fixed value | `LLIN-bi` |
| 15 | `administrative_area` | Yes | No | Must match a boundary name from `boundary_template.csv` | Cross-validated with `boundary_code` |
| 16 | `boundary_code` | Yes | No | Must match a boundary ID from `boundary_template.csv` | Cross-validated with `administrative_area` |

---

#### Cross-Field Validation
- **`boundary_code` + `administrative_area`**: The boundary code must exist in `boundary_template.csv`, the administrative area must match the name for that code. If they don't correspond to each other, validation fails.

#### Uniqueness Checks (within the CSV)
- **`username`** — no two rows can have the same username
- **`mobile_number`** — no two rows can have the same mobile number (empty values are ignored)

---

#### Valid Roles (from `rolesmapping.json`)
Roles are comma-separated. Each value must match one of:

---

### Quick Start: Run all cells below (Cell → Run All)

In [6]:
# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import pandas as pd
import os
from datetime import datetime
import sys
import glob
import time

# Force reload modules to pick up changes
import importlib
if 'utils' in sys.modules:
    importlib.reload(sys.modules['utils.api_client'])
    importlib.reload(sys.modules['utils.validators'])
    importlib.reload(sys.modules['utils.user_custom_validators'])
    importlib.reload(sys.modules['utils'])

# Import our custom modules
from utils import create_user_validator
from utils.api_client import APIClient

# Clear uploads folder on startup
for file in glob.glob("uploads/*"):
    if os.path.isfile(file):
        os.remove(file)
print(" All libraries loaded successfully!")
print(" Uploads folder cleared!")


 All libraries loaded successfully!
 Uploads folder cleared!


In [7]:
api_url_input = widgets.Text(
    value='http://hcm-moz-impl.abuja:8080/hcm-moz-impl/v1/dhis2/users/ingest?source=EXCEL',
    placeholder='Enter API URL',
    description='API URL:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='800px')
)
tenant_id_input = widgets.Text(
    value='bi',
    placeholder='Enter Tenant ID',
    description='Tenant ID:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)
auth_token_input = widgets.Text(
    value='ee36fdd7-64e7-4583-9c16-998479ff53c0',
    placeholder='Enter Auth Token',
    description='Auth Token:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='500px')
)

# ============================================
# FILE BROWSER FUNCTIONS
# ============================================
def scan_csv_files():
    """Scan for CSV files in common locations"""
    csv_files = []
    search_paths = [
        'uploads/*.csv'
    ]
    for pattern in search_paths:
        csv_files.extend(glob.glob(pattern))
    # Remove duplicates and sort
    csv_files = sorted(set(csv_files))
    return csv_files

def refresh_file_list(b=None):
    """Refresh the file dropdown with available CSV files"""
    csv_files = scan_csv_files()
    if csv_files:
        file_dropdown.options = [('-- Select a CSV file --', '')] + [(f, f) for f in csv_files]
    else:
        file_dropdown.options = [('No CSV files found', '')]
    file_dropdown.value = ''

# File dropdown selector
file_dropdown = widgets.Dropdown(
    options=[('-- Click Refresh to scan --', '')],
    value='',
    description='Select File:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px')
)

# Refresh button
refresh_button = widgets.Button(
    description='Refresh',
    button_style='info',
    icon='refresh',
    layout=widgets.Layout(width='100px')
)
refresh_button.on_click(refresh_file_list)

# Manual path input (alternative)
file_path_input = widgets.Text(
    value='',
    placeholder='Or type path manually: templates/users.csv',
    description='Manual Path:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='450px')
)

process_button = widgets.Button(
    description=' Process & Upload',
    button_style='primary',
    icon='upload',
    layout=widgets.Layout(width='250px', height='50px')
)
# Output for processing logs ONLY
process_output = widgets.Output()
status_label = widgets.HTML(value="<h3 style='color: #3498DB;'> Ready to process CSV</h3>")
# Global variables - MUST be initialized here
summary_data = None
# ============================================
# HELPER FUNCTION - Clear uploads folder
# ============================================
def clear_uploads_folder(exclude_file=None):
    """Clear all files in uploads folder except the specified file"""
    import glob
    for file in glob.glob("uploads/*"):
        if os.path.isfile(file):
            # Don't delete the source file we need to process
            if exclude_file and os.path.samefile(file, exclude_file):
                continue
            os.remove(file)

def resolve_file_path(path):
    """
    Resolve file path - handles both relative and absolute paths
    """
    path = path.strip()
    if os.path.isabs(path):
        return path
    return os.path.abspath(path)

def normalize_date(date_str):
    """
    Normalize date format: convert DD-MM-YYYY to DD/MM/YYYY for API compatibility
    """
    if pd.isna(date_str) or str(date_str).strip() == '' or str(date_str).lower() == 'nan':
        return date_str
    date_str = str(date_str).strip()
    # Replace hyphens with slashes
    return date_str.replace('-', '/')

def normalize_row_dates(row):
    """
    Normalize all date fields in a row for API upload
    """
    date_columns = ['date_of_joining', 'date_of_birth']
    for col in date_columns:
        if col in row.index:
            row[col] = normalize_date(row[col])
    return row

# ============================================
# EVENT HANDLER
# ============================================
def on_process_click(b):
    """
    Handle process button - validates then auto-uploads if all pass
    """
    global summary_data
    
    # Clear process output
    with process_output:
        clear_output()
    
    # Get file path - prefer dropdown, fallback to manual input
    input_path = file_dropdown.value if file_dropdown.value else file_path_input.value.strip()
    
    if not input_path:
        status_label.value = "<h3 style='color: red;'> Please select a CSV file or enter a path!</h3>"
        return
    
    # Resolve to absolute path
    csv_path = resolve_file_path(input_path)
    
    # Check if file exists
    if not os.path.exists(csv_path):
        status_label.value = f"<h3 style='color: red;'> File not found: {input_path}</h3>"
        with process_output:
            print(f"[ERROR] File not found")
            print(f"   Input path: {input_path}")
            print(f"   Resolved to: {csv_path}")
            print(f"\n[TIP] Click 'Refresh' to scan for available CSV files")
        return
    
    # Disable button during processing
    process_button.disabled = True
    status_label.value = "<h3 style='color: orange;'> Processing...</h3>"
    
    try:
        # Get filename from path
        uploaded_filename = os.path.basename(csv_path)
        upload_path = f"uploads/{uploaded_filename}"
        
        # Check if source is already in uploads folder
        source_in_uploads = csv_path.replace('\\', '/').find('/uploads/') != -1 or csv_path.startswith('uploads/')
        
        if source_in_uploads:
            # Source is already in uploads - use it directly, clear other files
            clear_uploads_folder(exclude_file=csv_path)
            upload_path = csv_path  # Use source directly
        else:
            # Source is outside uploads - clear folder and copy
            clear_uploads_folder()
            import shutil
            shutil.copy(csv_path, upload_path)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        
        # ============================================
        # PHASE 1: VALIDATION
        # ============================================
        with process_output:
            print("=" * 70)
            print("[PHASE 1] CSV VALIDATION")
            print("=" * 70)
            print(f"\n[INFO] File: {uploaded_filename}")
            print(f"[INFO] Source path: {csv_path}")
            print(f"[INFO] Working copy: {upload_path}")
        
        # Initialize validator
        validator = create_user_validator()
        
        with process_output:
            print(f"\n[VALIDATING] Running validation checks...")
        
        # Validate CSV
        validated_df, summary = validator.validate_csv(upload_path)
        
        with process_output:
            print(f"\n{'=' * 70}")
            print("[VALIDATION SUMMARY]")
            print(f"{'=' * 70}")
            print(f"[HEADER STATUS] {summary['header_status']}")
            if summary['header_status'] == 'ERROR':
                print(f"[HEADER ERROR] {summary['header_message']}")
            print(f"[TOTAL USERS] {summary['total_users']}")
            print(f"[ VALID USERS] {summary['correct_users']}")
            print(f"[ INVALID USERS] {summary['error_users']}")
            print(f"{'=' * 70}\n")
        
        # Check if ALL validations passed
        if summary['error_users'] > 0 or summary['header_status'] == 'ERROR':
            # VALIDATION FAILED
            base_name = uploaded_filename.rsplit('.', 1)[0]
            error_report_path = f"uploads/{base_name}_errors.csv"
            
            # Get only error rows
            error_df = validated_df[validated_df['validation_status'] == 'ERROR']
            error_df.to_csv(error_report_path, index=False)
            
            with process_output:
                print(f"[FAILED]  Validation failed with {summary['error_users']} errors\n")
                print(f" Updating error report: {error_report_path}")
                print(f"    Error report saved successfully!")
            
            status_label.value = f"<h3 style='color: red;'> Validation Failed: {summary['error_users']} errors. Run the Summary cell below!</h3>"
            
            # Store summary data for next cell
            summary_data = {
                'status': 'FAILED',
                'summary': summary,
                'error_report': error_report_path
            }
            
        else:
            # ALL VALIDATIONS PASSED
            with process_output:
                print(f"[SUCCESS]  All validations passed!\n")
                print(f"{'=' * 70}")
                print(f"[PHASE 2] API UPLOAD ")
                print(f"{'=' * 70}\n")
            
            # Save validated file in uploads (overwrite original)
            validated_df.to_csv(upload_path, index=False)
            
            # Initialize API client
            api_url = api_url_input.value
            tenant_id = tenant_id_input.value
            auth_token = auth_token_input.value
            
            with process_output:
                print(f"[UPLOADING] User Data")
                print(f"   Total Users: {summary['total_users']}")
                print(f"   API URL: {api_url}")
                print(f"   Tenant ID: {tenant_id}")
                print(f"   Strategy: CREATE mode")
                print(f"{'=' * 70}\n")
            
            client = APIClient(api_url, tenant_id, auth_token)
            
            # Process and upload row by row
            base_name = uploaded_filename.rsplit('.', 1)[0]
            final_report_path = f"uploads/{base_name}_result.csv"
            
            # Read validated CSV
            df = pd.read_csv(upload_path)
            
            # Initialize API response columns
            df['api_status'] = ''
            df['api_status_code'] = ''
            df['api_message'] = ''
            
            created_count = 0
            failed_count = 0
            skipped_count = 0
            
            # Only process rows with validation_status = 'CORRECT'
            for idx, row in df.iterrows():
                if row.get('validation_status') == 'CORRECT':
                    with process_output:
                        print(f"[PROCESSING] Row {idx + 1}/{len(df)}: {row.get('username', 'N/A')}", end='')
                    
                    # Normalize dates for API (convert - to /)
                    row_normalized = normalize_row_dates(row.copy())
                    
                    # Create temp CSV for single row
                    temp_file = f"temp_upload_{idx}.csv"
                    single_row_df = pd.DataFrame([row_normalized])
                    single_row_df.to_csv(temp_file, index=False)
                    
                    result = client.upload_file(temp_file)
                    
                    # Update DataFrame with API response
                    df.at[idx, 'api_status'] = result['status']
                    df.at[idx, 'api_status_code'] = str(result['status_code'])
                    df.at[idx, 'api_message'] = result['message']
                    
                    # Count results based on status
                    if result['status'] == 'CREATED':
                        created_count += 1
                        with process_output:
                            print(f" → ✓ CREATED (Status: {result['status_code']})")
                    else:  # FAILED
                        failed_count += 1
                        with process_output:
                            print(f" → ✗ FAILED (Status: {result['status_code']})")
                    
                    # Clean up temp file
                    if os.path.exists(temp_file):
                        os.remove(temp_file)
                    
                    # Delay between requests
                    time.sleep(5)
                else:
                    # Skip rows with validation errors
                    df.at[idx, 'api_status'] = 'SKIPPED'
                    df.at[idx, 'api_status_code'] = 'N/A'
                    df.at[idx, 'api_message'] = 'Validation failed'
                    skipped_count += 1
            
            # Save output
            df.to_csv(final_report_path, index=False)
            
            with process_output:
                print(f"\n{'=' * 70}")
                print("[API UPLOAD SUMMARY]")
                print(f"{'=' * 70}")
                print(f"[TOTAL UPLOADED] {len(df)}")
                print(f"[✓ CREATED] {created_count}")
                print(f"[✗ FAILED] {failed_count}")
                print(f"[⊘ SKIPPED] {skipped_count}")
                print(f"{'=' * 70}\n")
                print(f" Updating result file: {final_report_path}")
                print(f"    Result file updated successfully!")
                print(f"    Updated {len(df)} rows\n")
                print(" DATA UPLOAD COMPLETED!")
            
            success_count = created_count 
            status_label.value = f"<h3 style='color: green;'> Complete! {created_count} created, {failed_count} failed. Run Summary cell below!</h3>"
            
            # Store summary data for next cell
            summary_data = {
                'status': 'SUCCESS',
                'summary': summary,
                'created_count': created_count,
                'failed_count': failed_count,
                'final_report': final_report_path
            }
        
    except Exception as e:
        with process_output:
            print(f"\n Error: {str(e)}")
            import traceback
            traceback.print_exc()
        status_label.value = f"<h3 style='color: red;'> Process failed: {str(e)}</h3>"
    
    finally:
        # Re-enable button
        process_button.disabled = False

# Attach event handler
process_button.on_click(on_process_click)

# Initial file scan
refresh_file_list()

# ============================================
# DISPLAY UI
# ============================================
display(HTML("<hr><h2 style='color: #2E86C1;'> Configuration</h2>"))
display(widgets.VBox([
    api_url_input,
    widgets.HBox([tenant_id_input, auth_token_input]),
]))
display(HTML("<hr><h2 style='color: #2E86C1;'> CSV File</h2>"))
display(HTML("<p style='color: #666;'>Select from dropdown or type path manually:</p>"))
display(widgets.HBox([file_dropdown, refresh_button]))
display(file_path_input)
display(process_button)
display(status_label)
display(HTML("<hr><h2 style='color: #2E86C1;'> Processing Logs</h2>"))
display(process_output)
print("\n Application ready! Select a CSV file and click 'Process & Upload'")

Text(value='', description='Manual Path:', layout=Layout(width='450px'), placeholder='Or type path manually: t…

Button(button_style='primary', description=' Process & Upload', icon='upload', layout=Layout(height='50px', wi…

HTML(value="<h3 style='color: #3498DB;'> Ready to process CSV</h3>")

Output()


 Application ready! Select a CSV file and click 'Process & Upload'


---
##  Summary Report & Downloads

**Run the cell below after processing to see results:**

In [8]:
# Display Summary and Downloads
if 'summary_data' in globals() and summary_data:
    timestamp_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    if summary_data['status'] == 'FAILED':
        # Validation Failed
        summary = summary_data['summary']
        error_report = summary_data['error_report']
        
        display(HTML(f"""
        <div style='background-color: #FADBD8; padding: 20px; border-radius: 10px; border-left: 5px solid #E74C3C; margin: 20px 0;'>
            <h2 style='color: #E74C3C; margin-top: 0;'> VALIDATION FAILED</h2>
            <table style='width: 100%; border-collapse: collapse; background-color: white; margin-top: 15px;'>
                <tr style='background-color: #E74C3C; color: white;'>
                    <th style='padding: 10px; text-align: left;'>Metric</th>
                    <th style='padding: 10px; text-align: center;'>Count</th>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'>Total Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; font-weight: bold;'>{summary['total_users']}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'> Valid Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; color: green; font-weight: bold;'>{summary['correct_users']}</td>
                </tr>
                <tr>
                    <td style='padding: 8px; border: 1px solid #ddd;'> Invalid Users</td>
                    <td style='padding: 8px; border: 1px solid #ddd; text-align: center; color: red; font-weight: bold;'>{summary['error_users']}</td>
                </tr>
            </table>
            <p style='margin-top: 15px; font-weight: bold;'> Fix errors and re-upload</p>
        </div>
        <h3 style='color: #E74C3C;'> Download Error Report:</h3>
        <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #E74C3C; display: inline-block; margin-top: 10px;">
            <a href="{error_report}" download="{os.path.basename(error_report)}"
               style="display: inline-block; padding: 10px 20px; background-color: #E74C3C; color: white;
                      text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px;">
                 Download Error Report ({os.path.basename(error_report)})
            </a>
        </div>
        """))
        
    elif summary_data['status'] == 'SUCCESS':
        # API Upload Success
        summary = summary_data['summary']
        created_count = summary_data['created_count']
        failed_count = summary_data['failed_count']
        final_report = summary_data['final_report']
        
        display(HTML(f"""
        <div style="font-family: Arial, sans-serif; padding: 20px; border: 2px solid #007bff; border-radius: 10px; background-color: #f8f9fa; margin: 20px 0;">
            <h2 style="color: #007bff; margin-top: 0;"> Data Upload Summary Report</h2>
            <p style="color: #666; margin-bottom: 10px;">Generated: {timestamp_str}</p>
            
        
            
            <div style="display: flex; justify-content: space-around; margin: 20px 0; flex-wrap: wrap;">
                <div style="text-align: center; padding: 15px; background-color: #d4edda; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
                    <div style="font-size: 32px; font-weight: bold; color: #155724;">{created_count}</div>
                    <div style="color: #155724;">✓ Created</div>
                </div>
                <div style="text-align: center; padding: 15px; background-color: #f8d7da; border-radius: 5px; flex: 1; margin: 5px; min-width: 150px;">
                    <div style="font-size: 32px; font-weight: bold; color: #721c24;">{failed_count}</div>
                    <div style="color: #721c24;">✗ Failed</div>
                </div>
            </div>
        </div>
        """))
        
        status_badge = ' ALL SUCCESS' if failed_count == 0 else ' HAS ERRORS'
        badge_color = '#28a745' if failed_count == 0 else '#dc3545'
        
        display(HTML(f"""
        <div style="margin: 20px 0; padding: 15px; background-color: #e7f3ff; border-left: 4px solid #007bff; border-radius: 5px;">
            <h3 style="margin-top: 0; color: #004085;"> Updated CSV File with API Status</h3>
            <p style="color: #004085; margin-bottom: 15px;">
                The CSV file has been updated with <b>api_status</b>, <b>api_status_code</b>, and <b>api_message</b> columns:
            </p>
            <div style="padding: 10px; background-color: white; border-radius: 5px; border: 1px solid #ddd;">
                <div style="display: flex; justify-content: space-between; align-items: center; flex-wrap: wrap;">
                    <div style="flex: 1; min-width: 300px;">
                        <strong style="color: #007bff;"> {os.path.basename(final_report)}</strong>
                        <span style="background-color: {badge_color}; color: white; padding: 3px 8px; border-radius: 3px; font-size: 11px; margin-left: 10px;">{status_badge}</span>
                        <br>
                    </div>
                    <a href="{final_report}" download="{os.path.basename(final_report)}" 
                       style="display: inline-block; padding: 8px 16px; background-color: #007bff; color: white; 
                              text-decoration: none; border-radius: 5px; font-weight: bold; font-size: 14px; margin-top: 5px;">
                         Download Result File
                    </a>
                </div>
            </div>
            <p style="color: #004085; font-size: 12px; margin-top: 15px; border-top: 1px solid #bee5eb; padding-top: 10px;">
                <b> Status Values in CSV:</b>
            </p>
            <ul style="font-size: 12px; color: #004085; margin: 5px 0;">
                <li><span style="color: green; font-weight: bold;">CREATED:</span> New user created successfully and update if already exist</li>
                <li><span style="color: red; font-weight: bold;">FAILED:</span> Failed to create/update - check api_message column for details</li>
                <li><span style="color: gray; font-weight: bold;">SKIPPED:</span> Validation failed (not uploaded)</li>
            </ul>
        </div>
        """))

else:
    print(" No results yet. Please upload and process a CSV file in the cell above first.")